###### 7_lstm_practice.ipynb

## RNN (Recurrent Neural Network)
- 시퀀스(순서 있는 데이터)를 처리하는 신경망
- 이전 시점의 hidden state를 현재 입력과 함께 사용하여 '시간 흐름'의 패턴을 학습
- 아래와 같은 시간에 따른 변화가 있는 데이터에 적합:
  - 텍스트 (문장)
  - 타임시리즈 (주가, 센서 데이터)
  - 음성, 음악
- 단, 시퀀스 길이가 길어지면(시간이 길어지면) 과거 정보가 희미해지는 '장기 의존성 문제'가 발생

## LSTM (Long Short-Term Memory)
- RNN의 단점을 보완하기 위해 고안된 구조로, '게이트(gate)'라는 메커니즘을 사용하여 중요한 정보를 오래 기억할 수 있음
- 장기 의존성 문제를 해결하는 데 강하고, 자연어 처리(NLP)와 시계열 분야에서 매우 자주 사용됨

In [ ]:
import numpy as np
# * 데이터 준비

# x = [[1,2,3], [2,3,4], [3,4,5], ..., [47,48,49]]
# y = [4, 5, 6, ..., 50]
data = np.arange(1, 51)     # 1 ~ 50

x, y = [], []
window = 3

for i in range( len(data) - window ):   # range(50-3) => range(47)
  x.append( data[i:i+window] )    # i=0) data[0:3] -> [1,2,3]
  y.append( data[i+window] )      # i=0) data[3] -> 4

# print(x, y)
# list -> ndarray
x = np.array(x)
y = np.array(y)

# print(type(x), type(y))
print(x.shape)

# * RNN / LSTM => (batch, timesteps, features)
x = x.reshape(x.shape[0], x.shape[1], 1)
# => 한 시점에 값이 1개인 1차원 데이터

(47, 3)
